In [ ]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib

nb_dir = (
    pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
)
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import (
    SeqOperator,
    PrimitiveEventStructure,
    KleeneClosureOperator,
)
from condition.CompositeCondition import AndCondition
from condition.Condition import Variable, SimpleCondition
from condition.KCCondition import KCIndexCondition
from stream.FileStream import FileOutputStream
from stream.DataFrameStream import CitiBikeDataFrameInputStream
from plugin.citibike.CitiBike2 import (
    CitiBikeTripEventTypeClassifier,
    CitiBikeDataFormatter,
)
import test
from tree.PatternMatchStorage import TreeStorageParameters
from parallel.ParallelExecutionParameters import (
    DataParallelExecutionParametersHirzelAlgorithm,
)
from parallel.ParallelExecutionPlatforms import ParallelExecutionPlatforms
from plugin.citibike.RessourceConsumption import RessourceConsumption

monitor = RessourceConsumption()


In [ ]:
citibikeHotPathsPattern = Pattern(
    SeqOperator(
        KleeneClosureOperator(PrimitiveEventStructure("CitiBikeTrip", "a"), max_size=5),
        PrimitiveEventStructure("CitiBikeTrip", "b"),
    ),
    AndCondition(
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: x["bikeid"],
            relation_op=lambda a1, a2: a1 == a2,
            offset=-1,
        ),
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: (
                int(float(x["startstationid"])),
                int(float(x["endstationid"])),
            ),
            relation_op=lambda a1, a2: a1[0] == a2[1],
            offset=-1,
        ),
        SimpleCondition(
            Variable("a", lambda x: x[-1]["bikeid"]),
            Variable("b", lambda x: x["bikeid"]),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("a", lambda x: int(float(x[-1]["endstationid"]))),
            Variable("b", lambda x: int(float(x["startstationid"]))),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("b", lambda x: int(float(x["endstationid"]))),
            relation_op=lambda end_id: str(end_id) in {"111111"},
        ),
    ),
    timedelta(minutes=31),
)

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-small.csv"
output_file = "output_citibike_baseline_small_try.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)
cep = CEP([citibikeHotPathsPattern])
monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Baseline - small dataset

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-small.csv"
output_file = "output_citibike_baseline_small.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)
cep = CEP([citibikeHotPathsPattern])
monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Both small dataste

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-small.csv"
output_file = "output_citibike_both_small.txt"

load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=30,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.08
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Both half dataset

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-original-half.csv"
output_file = "output_citibike_both_half2.txt"

load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=30,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.08
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Both full dataset- max() latency

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-original.csv"
output_file = "output_citibike_both_FULL_DATASET.txt"

load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=30,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.08
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Both full dataset, new latency and without weird if

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-original.csv"
output_file = "output_citibike_both_FULL_DATASET-fixed?.txt"

load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=30,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.08
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

In [ ]:
# Parallelization only - big dataset

In [ ]:
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params)

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated.csv"
output_file = "output_citibike_only_parallel.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)


In [ ]:
# cep.run(
#     events,
#     FileOutputStream("../test/demo/Matches/testing", output_file),
#     CitiBikeDataFormatter(),
# )
#ran for 513 min

# Parallelization only - small dataset

In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-small.csv"
output_file = "output_citibike_only_parallel-small.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

In [ ]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Parallelization & Load shedding

In [ ]:
load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=15,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=8,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )


In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated.csv"
output_file = "output_citibike_both.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

In [ ]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)# ran for an hour

# Load shedding 2

In [ ]:
load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=15,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=8,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )


In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated.csv"
output_file = "output_citibike_both-2.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

In [ ]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)

# Both after merge

In [ ]:
load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=15,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.01
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=8,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )


In [ ]:
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated-small.csv"
output_file = "output_citibike_both-merge.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)

In [ ]:
monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches", output_file),
    CitiBikeDataFormatter(),
)

# Both big dataset

In [ ]:
load_shedding_params = TreeStorageParameters(
    sort_storage=True,
    enable_load_shedding=True,
    load_shedding_threshold=50,
    load_shedding_drop_rate=0.3,
    load_shedding_strategy="oldest",
    clean_up_interval=10,
    latency_bound=0.1
)
dp_params = DataParallelExecutionParametersHirzelAlgorithm(
    platform=ParallelExecutionPlatforms.THREADING,
    units_number=12,  # how many threads
    key="bikeid",
)
cep = CEP([citibikeHotPathsPattern], parallel_execution_params=dp_params, storage_params=load_shedding_params )
input_file = "../test/EventFiles/201901-citibike-tripdata-1-fabricated.csv"
output_file = "output_citibike_both-merge-big-0.1latency.txt"

events = CitiBikeDataFrameInputStream(
    input_file,
    timestamp_column="starttime",
)


In [ ]:
monitor.run(
    cep.run,
    events,
    FileOutputStream("../test/demo/Matches/testing", output_file),
    CitiBikeDataFormatter(),
)